In [2]:
#require: pandas, tensorflow_hub, tensorflow_text, tensorflow_addons, sklearn
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
import tensorflow_text as text  # Imports TF ops for preprocessing.
import model.tokenization as tokenization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping,CSVLogger
from tensorflow.keras.layers import Input, Dense,Dropout,Embedding,LSTM,Bidirectional, Masking, TimeDistributed, Conv1D, MaxPooling1D, Flatten, concatenate, GRU

random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)
BERT_src = 'https://tfhub.dev/tensorflow/bert_zh_L-12_H-768_A-12/3'#'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4' 
BERT_LAYER = hub.KerasLayer(BERT_src, trainable=False)

In [3]:
category = ['功能', '品質', '無', '配件', '售後', '外觀', '價位', '音量']
cate2idx = {cate:idx for idx, cate in enumerate(category)}
idx2cate = {idx: cate for cate, idx in cate2idx.items()}
sentiment = ['負向', '正向', '中立']
sent2idx = {sent:idx for idx, sent in enumerate(sentiment)}
idx2sent = {idx: sent for sent, idx in sent2idx.items()}

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if len(gpus)!=0:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)
else:
    print("No GPUs visible")

Physical devices cannot be modified after being initialized


# Read Data

In [4]:
def twoSentence2BERT(inputs, target_, onlytarget=False): #input as list of dictionary
#     BERT_LAYER = hub.KerasLayer(BERT_src, trainable=False)
    VOCAB_FILE = BERT_LAYER.resolved_object.vocab_file.asset_path.numpy()
    tokenizer = tokenization.FullTokenizer(VOCAB_FILE, True) 
    output={'input_word_ids':[], 'input_mask':[], 'input_type_ids':[]}
    for data in inputs:
        count = 0
        #tokenize origin sentence
        tempword, tempmask, temptype=[], [], []
        #add cls
        tempword.append(tokenizer.convert_tokens_to_ids(['[CLS]'])[0])
        tempmask.append(1)
        temptype.append(0)
        if not onlytarget:
            sentence = data['sentence'] #string
            tokenize_sentence = tokenizer.tokenize(sentence)
            for ts in tokenize_sentence:
                try:
                    token_id = tokenizer.convert_tokens_to_ids([ts.lower()])
                except:
                    token_id = tokenizer.convert_tokens_to_ids(['[UNK]'])
                tempword.append(token_id[0])
                tempmask.append(1)
                temptype.append(0)
                count+=1
            #add sep
            if target_:
                tempword.append(tokenizer.convert_tokens_to_ids(['[SEP]'])[0])
                tempmask.append(1)
                temptype.append(0)
        if target_:
            target = data['target'] #string        
            tokenize_target = tokenizer.tokenize(target)        
            for tt in tokenize_target:
                try:
                    token_id = tokenizer.convert_tokens_to_ids([tt.lower()])
                except:
                    token_id = tokenizer.convert_tokens_to_ids(['[UNK]'])
                tempword.append(token_id[0])
                tempmask.append(1)
                temptype.append(1)
                count+=1
                if count>=128:
                    break
        if len(tempword)>127:
            tempword=tempword[:127]
            tempmask=tempmask[:127]
            temptype=temptype[:127]  
        #add sep
        tempword.append(tokenizer.convert_tokens_to_ids(['[SEP]'])[0])
        tempmask.append(1)
        temptype.append(1)                
        while(len(tempword)<128):
            tempword.append(0)
            tempmask.append(0)
            temptype.append(0)            
        output['input_word_ids'].append(tempword)
        output['input_mask'].append(tempmask)
        output['input_type_ids'].append(temptype)        
    return output

def BERTdata2Traindata(data, target=True, to_cate=True, objective = ['sentiment', 'aspect_category']):
    from tensorflow.keras.utils import to_categorical
    outputx = twoSentence2BERT(data, target)
    outputy_sentiment = []
    outputy_category = []
    category=True
    if 'aspect_category' not in objective:
        category=False
        
    if to_cate:
        for d in data:
            if 'sentiment' in objective:
                outputy_sentiment.append(sent2idx[d['sentiment']])
            elif 'aspect_category' in objective:
                outputy_category.append(cate2idx[d['aspect_category']])
    else:
        for d in data:
            if 'sentiment' in objective:
                outputy_sentiment.append(d['sentiment'])
            elif 'aspect_category' in objective:
                outputy_category.append(d['aspect_category'])
        
    if category:
        if to_cate:
            return outputx, to_categorical(outputy_sentiment, num_classes=len(sent2idx)), to_categorical(outputy_category, num_classes=len(cate2idx))
        else:
            return outputx, outputy_sentiment, outputy_category
            
    else:
        if to_cate:
            return outputx, to_categorical(outputy_sentiment, num_classes=len(sent2idx))
        else:
            return outputx, outputy_sentiment
def transBERTtype(data, toBERT=True):
    if toBERT: #input 每個資料都有三個key，每個key的維度都是128
        return {k:np.array([data[i][k] for i in range(len(data))]) for k in data[0].keys()}
    else: #原本BERT的形式
        return [{k:data[k][i] for k in data.keys()} for i in range(len(data['input_word_ids']))]        

In [5]:
# #0:負向；1:正向；2:中立
singer_chenyu_train = pd.read_csv('./data/chenyu_ABSA/dev.tsv', sep='\t', header=0) #1238
singer_chenyu_train.columns = ['sentiment', 'target', 'sentence']
singer_chenyu_train = list(singer_chenyu_train.T.to_dict().values())
for dta in range(len(singer_chenyu_train)):
    singer_chenyu_train[dta]['sentiment']=idx2sent[singer_chenyu_train[dta]['sentiment']]

In [6]:
import json
BERT_train, BERT_test = [], []
restaurant, laptop, singer = [], [], []
with open('./data/homeapp/PB-ACSent_BERT_train_0414.json', 'r', encoding='utf8') as file:
    data = file.readlines()
for d in data:
    BERT_train.append(json.loads(d))
    
with open('./data/homeapp/PB-ACSent_BERT_test_0414.json', 'r', encoding='utf8') as file:
    data = file.readlines()
for d in data:
    BERT_test.append(json.loads(d))

with open('./data/translate/filt_restaurant.json', 'r', encoding='utf8') as file:
    data = file.readlines()
for d in data:
    restaurant.append(json.loads(d))    
    
with open('./data/translate/filt_laptop.json', 'r', encoding='utf8') as file:
    data = file.readlines()
for d in data:
    laptop.append(json.loads(d))    
    
with open('./data/Singer/SingerABSA.json', 'r', encoding='utf8') as file:
    data = file.readlines()
for d in data:
    singer.append(json.loads(d))    
    
print('train:{}, test:{}'.format(len(BERT_train), len(BERT_test)))
print('restuarant:{}, laptop:{}, singer:{}'.format(len(restaurant), len(laptop), len(singer)))

train:1663, test:1725
restuarant:1908, laptop:1220, singer:6326


In [6]:
#source
restaurant_x, restaurant_sentiment = BERTdata2Traindata(restaurant, to_cate=True, objective=['sentiment'])
laptop_x, laptop_sentiment = BERTdata2Traindata(laptop, to_cate=True, objective=['sentiment'])
singer_chenyu_x, singer_chenyu_sentiment = BERTdata2Traindata(singer_chenyu_train, to_cate=True, objective=['sentiment'])
singer_x, singer_sentiment = BERTdata2Traindata(singer, to_cate=True, objective=['sentiment'])
#target
# train_x, train_sentiment, train_category = BERTdata2Traindata(BERT_train)
# test_x, test_sentiment, test_category = BERTdata2Traindata(BERT_test)
train_x, train_sentiment = BERTdata2Traindata(BERT_train, to_cate=True, objective=['sentiment'])
test_x, test_sentiment = BERTdata2Traindata(BERT_test, to_cate=True, objective=['sentiment'])

# Model

In [8]:
#Gradient Reverse Layer
@tf.custom_gradient
def grad_reverse(x):
    y = tf.identity(x)
    def custom_grad(dy):
        return -dy
    return y, custom_grad

class GradReverse(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def call(self, x):
        return grad_reverse(x)

In [9]:
#sentence base, multi-task approach
def create_classify_model(data_size, batch_size = 16, epochs=10):
    import model.optimization as optimization
    input1 = Input(shape=(128,), name='input_word_ids', dtype=tf.int32)
    input2 = Input(shape=(128,),name='input_mask', dtype=tf.int32)
    input3 = Input(shape=(128,),name='input_type_ids', dtype=tf.int32)
    bert_layer = hub.KerasLayer(BERT_src, trainable=True, output_key='pooled_output', name='bert_layer')
    output = bert_layer({'input_word_ids':input1, 'input_mask':input2, 'input_type_ids':input3})
#     output = Dense(128, name = 'presentation_')(output)
    
    sentiment_output = Dense(64, activation='relu', name = 'sentiment_pre', 
                             kernel_initializer=keras.initializers.glorot_normal(0), bias_initializer='zeros')(output)
#     sentiment_output = Dropout(0.2, name='sentiment_drop')(sentiment_output)
    sentiment_output = Dense(3, activation='softmax', name = 'sentiment', 
                             kernel_initializer=keras.initializers.glorot_normal(0), bias_initializer='zeros')(sentiment_output) #softmax會讓所有的output總和=1

    category_output = Dense(128, activation='relu', name = 'category_pre', 
                             kernel_initializer=keras.initializers.glorot_normal(0), bias_initializer='zeros')(output)    
#     category_output = Dropout(0.2, name='category_drop')(category_output)
    category_output = Dense(8, activation='softmax', name = 'category', 
                             kernel_initializer=keras.initializers.glorot_normal(0), bias_initializer='zeros')(category_output) #softmax會讓所有的output總和=1
    
    output_model = Model(inputs = [input1, input2, input3], outputs = [sentiment_output, category_output])
    optimizer = optimization.create_optimizer(
    5e-5, (data_size//batch_size)*epochs, int((epochs*data_size*0.1)//batch_size), 0.0, 'lamb')
    
    output_model.compile(optimizer=optimizer, 
                         loss={'sentiment':'categorical_crossentropy', 'category':'categorical_crossentropy'},
                         loss_weights={'sentiment':1., 'category':1})
    return output_model

In [17]:
#sentence base, only sentence approach
def create_temp_model(data_size, batch_size = 16, epochs=10, domain_size = 1):
    import model.optimization as optimization
    input1 = Input(shape=(128,), name='input_word_ids', dtype=tf.int32)
    input2 = Input(shape=(128,),name='input_mask', dtype=tf.int32)
    input3 = Input(shape=(128,),name='input_type_ids', dtype=tf.int32)
    bert_layer = hub.KerasLayer(BERT_src, trainable=True, output_key='pooled_output', name='bert_layer')
    output = bert_layer({'input_word_ids':input1, 'input_mask':input2, 'input_type_ids':input3})
#     output = Dense(128, name = 'presentation_')(output)
    
    sentiment_output = Dense(64, activation='relu', name = 'sentiment_pre', 
                             kernel_initializer=keras.initializers.he_normal(0), bias_initializer='zeros')(output)
#     sentiment_output = Dropout(0.2, name='sentiment_drop')(sentiment_output)
    sentiment_output = Dense(3, activation='softmax', name = 'sentiment', 
                             kernel_initializer=keras.initializers.he_normal(0), bias_initializer='zeros')(sentiment_output) #softmax會讓所有的output總和=1
    if domain_size>1:
        temp_output = GradReverse()(output)
        dis_output = Dense(64, activation='relu', name = 'dis_pre', 
                             kernel_initializer=keras.initializers.he_normal(0), bias_initializer='zeros')(temp_output)
        dis_output = Dense(domain_size, activation='softmax', name = 'discriminator', 
                             kernel_initializer=keras.initializers.he_normal(0), bias_initializer='zeros')(dis_output) #softmax會讓所有的output總和=1

    optimizer = optimization.create_optimizer(
    5e-5, (data_size//batch_size)*epochs, int((epochs*data_size*0.1)//batch_size), 0.0, 'adamw')
    if domain_size>1:        
#         decoding_model = Model(inputs = [input1, input2, input3], outputs = sentiment_output)
#         decoding_model.compile(optimizer=optimizer, loss=categorical_crossentropy)
#         discriminator_model = Model(inputs = [input1, input2, input3], outputs = dis_output)
#         discriminator_model.compile(optimizer=optimizer, loss=categorical_crossentropy)        
        output_model = Model(inputs = [input1, input2, input3], outputs = [sentiment_output, dis_output])
        final_model = Model(inputs = [input1, input2, input3], outputs = sentiment_output) 
        output_model.compile(optimizer=optimizer, 
                             loss={'sentiment':'categorical_crossentropy', 'discriminator':'categorical_crossentropy'},
                            loss_weights={'sentiment':1., 'discriminator':.3})
    else:
        output_model = Model(inputs = [input1, input2, input3], outputs = sentiment_output)
        final_model = Model(inputs = [input1, input2, input3], outputs = sentiment_output) 
        output_model.compile(optimizer=optimizer, 
                             loss={'sentiment':'categorical_crossentropy'})
        
    return output_model, final_model

## no BERT

In [140]:
import platform
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter
import preprocessing as preprocess
idx2word, word2idx,char_embeddings = preprocess.get_pretrain_char_emb('CKIP')
special_words = ['<PAD>', '<UNK>'] # 特殊词表示
with open('/tf/notebooks/code/old/zh-nlp-demo/data/char_vocabs.txt', "r", encoding="utf8") as fo:
    char_vocabs = [line.strip() for line in fo]
# char_vocabs = []
# for i in range(0x4e00,0x9fa6):
#     char_vocabs.append(chr(i))
char_vocabs = special_words + char_vocabs
char_vocabs = list(idx2word.values())
char_vocabs.sort()

def create_nobert_model(tags = 8, embed_size = 50, pretrain_emb=True): #compile first
    inputs = Input(shape=(128,), name='encoding_input')
    output = Masking(mask_value=word2idx['<PAD>'], name='encoding_mask')(inputs)
    
    tgt_inputs = Input(shape=(10,), name='target_input')
    tgt_output = Masking(mask_value=word2idx['<PAD>'], name='tgt_mask')(tgt_inputs)
    
    encoding = concatenate([output, tgt_output], axis=1)
    
    if pretrain_emb:
        embed_size = len(char_embeddings['我'])
        embedding_matrix = np.zeros((len(word2idx), embed_size))
        for word, i in word2idx.items():
            embedding_vector = char_embeddings.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector
        encoding = Embedding(len(char_vocabs), embed_size, weights=[embedding_matrix], trainable=True, name='encoding_emb')(encoding)  
    else:
        encoding = Embedding(len(char_vocabs), embed_size, trainable=True, name='encoding_emb')(encoding)
#         output = Embedding(len(char_vocabs), embed_size, trainable=True, name='encoding_emb')(output)  
#         tgt_output = Embedding(len(char_vocabs), embed_size, trainable=True, name='tgt_emb')(tgt_output)  
    
#    decoding_output = Dense(tags, activation='softmax', name='output')(encoding)

    cnn = Conv1D(1, kernel_size=1, strides=1,activation='relu', name='dis_conv')(encoding)
    cnn = MaxPooling1D(10, name='dis_maxpooling1d')(cnn)
    flat = Flatten(name='dis_flatten')(cnn)
    
    decoding_output = Dense(tags, activation='sigmoid', name='output')(flat)
    
    model = Model([inputs, tgt_inputs], decoding_output)
    model.compile(optimizer="adam",loss = 'binary_crossentropy', metrics=['accuracy'])

    return model

model = create_nobert_model()
model.summary()

In [159]:
import json
#PB-Sent_BERT_same_aux_NLI-B_train06-16.json
type_ = []
BERT_train, BERT_test = [], []
with open('../homeapp/LabelAPP/PB-AC_BERT_same_aux_NLI-M_train06-16.json', 'r', encoding='utf8') as file:
    data = file.readlines()
for d in data:
    BERT_train.append(json.loads(d))
    
with open('../homeapp/LabelAPP/PB-AC_BERT_same_aux_NLI-M_test06-16.json', 'r', encoding='utf8') as file:
    data = file.readlines()
for d in data:
    BERT_test.append(json.loads(d))

print('train:{}, test:{}'.format(len(BERT_train), len(BERT_test)))

train:1001, test:1075


In [182]:
def trans(data, maxlen=128):
    x = [np.array([word2idx.get(row, word2idx['<UNK>']) for row in s]) for s in data]
    x = np.array(pad_sequences(x, maxlen=maxlen, value=word2idx['<PAD>'], padding='post', truncating="post"))
    return x
sentidx ={'中立':0, 
          '正向':1, 
          '負向':2}
cateidx = {category[i]:i for i in range(len(category))}
sentence = {'train':trans([i['sentence'] for i in BERT_train]),
           'test':trans([i['sentence'] for i in BERT_test])}
target = {'train':trans([i['target'] for i in BERT_train], 10),
           'test':trans([i['target'] for i in BERT_test], 10)}
cate = {'train':  [[1 if category[j] in i['aspect_category'] else 0 for j in range(8)] for i in BERT_train],
    'test':  [[1 if category[j] in i['aspect_category'] else 0 for j in range(8)] for i in BERT_test]    }
# sentiment = {'train':  to_categorical([sentidx[i['sentiment']] for i in BERT_train]),
#              'test':  to_categorical([sentidx[i['sentiment']] for i in BERT_test])    
#             }
# sentiment = {'train':  np.array([sentidx[i['sentiment']] for i in BERT_train]),
#              'test':  np.array([sentidx[i['sentiment']] for i in BERT_test])    
#             }

In [185]:
history = model.fit(x={'encoding_input':sentence['train'], 'target_input':target['train']}, 
                    y=np.array(cate['train']), batch_size=16, epochs=7, verbose=1)
pred = model.predict([sentence['test'], target['test']])
# pred = [np.argmax(i) for i in pred]
# ans = [np.argmax(i) for i in sentiment['test']]
from sklearn.metrics import f1_score, accuracy_score, classification_report
print(accuracy_score(ans, pred))
print(classification_report(ans, pred))

Epoch 1/7
63/63 [==============================] - 2s 27ms/step - loss: 0.6662 - accuracy: 0.1849
Epoch 2/7
63/63 [==============================] - 2s 26ms/step - loss: 0.5236 - accuracy: 0.2487
Epoch 3/7
63/63 [==============================] - 2s 25ms/step - loss: 0.4289 - accuracy: 0.2604
Epoch 4/7
63/63 [==============================] - 2s 25ms/step - loss: 0.3984 - accuracy: 0.3296
Epoch 5/7
63/63 [==============================] - 2s 24ms/step - loss: 0.3886 - accuracy: 0.3602
Epoch 6/7
63/63 [==============================] - 2s 25ms/step - loss: 0.3728 - accuracy: 0.4343
Epoch 7/7
63/63 [==============================] - 2s 25ms/step - loss: 0.3667 - accuracy: 0.4266


ValueError: Found input variables with inconsistent numbers of samples: [1422, 1075]

# 其他設定

In [29]:
import random
def sample_data(data_list, datasize, random_=True):
    #data_list = [BERT_x, sentiment, (category)]
    if random_:
        samples = random.sample(range(len(data_list[1])), datasize)
    else:
        samples = list(range(datasize))        
    bert_x = data_list[0]
    bert_x = {k:np.array([bert_x[k][i] for i in samples]) for k in bert_x.keys()}
    sentiment = np.array(data_list[1])
    sentiment = np.array([sentiment[i] for i in samples])
    if len(data_list)>2:
        category = np.array(data_list[2])
        category = np.array([category[i] for i in samples])
        return bert_x, sentiment, category
    else:
        return bert_x, sentiment

In [30]:
def model_get_weight(model, keyword='', not_=False):
    origin_weight = []
    for layer in model.layers:
        if not_:
            if not layer.name.startswith(keyword): 
                origin_weight.append(np.array(layer.get_weights()))
        else:
            if layer.name.startswith(keyword): 
                origin_weight.append(np.array(layer.get_weights()))
    return np.array(origin_weight)

def update_weights(model, update_weight, keyword='', not_=False):
    k=0
    for layer in model.layers:
        if not_:
            if not layer.name.startswith(keyword):
                layer.set_weights(update_weight[k])
                k+=1
        else:
            if layer.name.startswith(keyword):
                layer.set_weights(update_weight[k])
                k+=1
def update_weights_forsame(model, model_src):
    for layer in model.layers:
        flag = False
        for layer_src in model_src.layers:
            if layer.name==layer_src.name and len(layer.get_weights())==len(layer_src.get_weights()) and flag==False:
                try: 
                    layer.set_weights(layer_src.get_weights())
                    flag = True
                except:
                    print('error!')
        if flag==False:
            print('model layer: "', layer.name, '" not in source model')
        

# Adversarial Reptile

In [13]:
src_x = [singer_x, restaurant_x, laptop_x]
src_y = [singer_sentiment, restaurant_sentiment, laptop_sentiment]
dev_x, dev_y = sample_data([singer_chenyu_x, singer_chenyu_sentiment], 300)

In [18]:
#這邊有一點比較遺憾的是，在reptile的實驗中，outer iteration通常都是設十萬以上
try:
    del tmp_model
except:
    ;
from sklearn import metrics
from sklearn.metrics import f1_score, accuracy_score, classification_report
outer_iteration = 50 #一個iteration大概3.1秒(一個domain)
inner_iteration = 3
batch = 5
epochs = inner_iteration
datasize_per_task = 32
meta_step_size = 1
domain_adver = len(src_y)
import time
starttime = time.time()
tmp_model, output_model = create_temp_model(data_size=len(singer_sentiment), epochs=epochs, batch_size=datasize_per_task, domain_size = domain_adver)
for itr in range(outer_iteration):
    if itr%10==0:
        print('itr =',itr)
    done_step = itr/outer_iteration
    cur_meta_step_size = (1-done_step)*meta_step_size
    origin_weights = model_get_weight(tmp_model)
    new_weights = []
    losses = []
    advloss = []
    adv_loss = 0
    domain_num = 0
    for x, y in zip(src_x, src_y):     
        loss = []
        for i in range(inner_iteration):
            if domain_adver>1:
                tmp_train_x, tmp_sentiment = sample_data([x, y], datasize_per_task*batch)  
                domain_ans = to_categorical(len(tmp_sentiment)*[domain_num], num_classes=domain_adver)
#                 total_loss = tmp_model.train_on_batch(x=tmp_train_x, y=[tmp_sentiment, domain_ans])                
#                 loss = round(total_loss[1], 5) 
#                 adv_loss = round(total_loss[2], 5)
                loss, adv_loss = 0.0, 0.0
                tmp_model.fit(tmp_train_x,[tmp_sentiment, domain_ans],
                            batch_size=datasize_per_task, epochs=inner_iteration, verbose=0) #一個batch會更新一次參數
                #這邊因為模型裡面本來就有gradient reverse layer，所以可以直接訓練沒問題
            else:
                tmp_train_x, tmp_sentiment = sample_data([x, y], datasize_per_task)  
                loss = tmp_model.train_on_batch(x=tmp_train_x, y=tmp_sentiment)
                loss = round(loss, 5)

        new_weights.append(model_get_weight(tmp_model))
        update_weights(tmp_model, origin_weights)
        losses.append(loss)
        advloss.append(adv_loss) #只看最後一個的loss好了
        domain_num+=1
    #update weights
    new_weights = np.array(new_weights)
    new_weight = new_weights[0]
    for i in range(len(new_weights)-1):
        new_weight+=new_weights[i+1]
    new_weight/=len(new_weights)    
    new_weight = origin_weights + ((new_weight-origin_weights)*cur_meta_step_size)
#     print('new', new_weight[3][0][0][0], 'origin',origin_weights[3][0][0][0], 'lr',round(cur_meta_step_size, 5))
    if domain_adver>1:
        pred = tmp_model.predict(dev_x)[0]
    else:
        pred = tmp_model.predict(dev_x)
        
    pred = [np.argmax(i) for i in pred]
    ans = [np.argmax(i) for i in dev_y]
    print('---------{}-------------'.format(accuracy_score(ans, pred)))
    
    print('new', new_weight[-1][0][0], 'origin',origin_weights[-1][0][0], 'lr',round(cur_meta_step_size, 5))
#     print('lr', round(cur_meta_step_size, 5), '\tloss', losses, '\t adv loss', round(np.mean(advloss), 5), '\t spend', int(time.time()-starttime))
    print('total spend {} seconds'.format(int(time.time()-starttime)))
    update_weights(tmp_model, new_weight)
    del new_weight, origin_weights, new_weights
print('total spend {} seconds'.format(int(time.time()-starttime)))
# output_model.save('rep_adv_opt_lamb.h5')

itr = 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.


---------0.43-------------
new [-0.3423988  -0.09649575 -0.13309965] origin [-0.34104967 -0.09674136 -0.1330885 ] lr 1.0
total spend 89 seconds


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.


---------0.57-------------
new [-0.3425071  -0.09662279 -0.13293625] origin [-0.3423988  -0.09649575 -0.13309965] lr 0.98
total spend 156 seconds


...

---------0.6133333333333333-------------
new [-0.34250748 -0.09665912 -0.13290295] origin [-0.34250748 -0.09665912 -0.13290295] lr 0.06
total spend 3441 seconds


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.


---------0.6133333333333333-------------
new [-0.34250748 -0.09665912 -0.13290295] origin [-0.34250748 -0.09665912 -0.13290295] lr 0.04
total spend 3513 seconds


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.


---------0.6133333333333333-------------
new [-0.34250748 -0.09665912 -0.13290295] origin [-0.34250748 -0.09665912 -0.13290295] lr 0.02
total spend 3585 seconds
total spend 3585 seconds
